## https://github.com/google-ai-edge/mediapipe/blob/master/docs/solutions/pose.md

In [4]:
import cv2
import mediapipe as mp
import csv

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# For webcam input:
cap = cv2.VideoCapture(0)

# Open a CSV file to save the joint coordinates
with open('pose_landmarks.csv', mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    # Write the header
    csv_writer.writerow(['Frame'] + [f'{joint}_{coord}' for joint in range(33) for coord in ['x', 'y', 'z', 'visibility']])

    with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
        frame_count = 0
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            # Check if pose landmarks are detected
            if results.pose_landmarks:
                # Extract landmarks
                landmarks = results.pose_landmarks.landmark
                row = [frame_count] + [coord for lm in landmarks for coord in [lm.x, lm.y, lm.z, lm.visibility]]
                csv_writer.writerow(row)

            # Draw the pose annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))

            if cv2.waitKey(5) & 0xFF == 27:  # Exit on pressing 'Esc'
                break

            frame_count += 1

cap.release()
cv2.destroyAllWindows()
